In [265]:
import math
import time
import tdmclient.notebook
import asyncio

await tdmclient.notebook.start()   





In [264]:
await tdmclient.notebook.stop()

In [266]:
# Get the first Thymio node
node = (await tdmclient.notebook.get_nodes())[0]


print("Connected to:", node._id if hasattr(node, "_id") else node)


Connected to: Node ff067687-6959-4893-b0ab-04a9b9fd9a6f


In [9]:
import sys, os

# Path to the project root (one level up from Core/)
project_root = os.path.abspath("..")

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Added to path:", project_root)


Added to path: c:\Users\danie\OneDrive\Documentos\FCUL\MobileRobots\ThymioBuilder


In [267]:
def wait_until_performed(var_name="on_performed", timeout=10):

    start = time.time()
    while time.time() - start < timeout:
        # Read variable from robot
        (value,) = get_var(var_name)

        if value is not None:

            if value == 1:
                print("✔ Action queue finished!")
                return True
            
            if value == -1:
                print("❌ Action queue encountered an error")
                return False

        time.sleep(0.1)  # avoid busy loop

    print("❌ Timeout waiting for action_queue to finish")
    return None

In [268]:
PRIMITIVE_MAP = {
    "F": [6],          # Forward
    "B": [1],          # Backwards full
    "HF": [2],         # Half forward
    "HB": [3],         # Half backwards
    "TL": [2, 5, 3],   # Turn left = half forward + rotate left + half back
    "TR": [2, 4, 3],   # Turn right = half forward + rotate right + half back
    "AB": [7, 8],          # Align Block +  Approach Block
}


def expand_actions(action_list):
    expanded = []
    for act in action_list:
        if act not in PRIMITIVE_MAP:
            print("Unknown action:", act)
            continue
        expanded.extend(PRIMITIVE_MAP[act])
    return expanded

EVENT_SIZE = 10

def make_payloads(action_list):
    expanded = expand_actions(action_list)
    payloads = []
    for i in range(0, len(expanded), EVENT_SIZE):
        chunk = expanded[i:i+EVENT_SIZE]
        chunk += [0] * (EVENT_SIZE - len(chunk))  # pad if shorter
        payloads.append(chunk)
    return payloads

async def send_action_queue_event(action_list):
    payloads = make_payloads(action_list)
    for idx, payload in enumerate(payloads):
      
        print(f"Sending payload {idx+1}/{len(payloads)}:", payload)
      
        await node.send_events({"action_queue": payload})
        # wait for robot confirmation before next payload
        wait_until_performed(var_name="on_performed", timeout=20)


In [269]:
import importlib
from Core.ActionQueue import ActionQueue
from Environment.Block_Manager import BlockManager
from Environment.Grid_Map import GridMap
from PathPlanner import PathPlanner


block_manager = None
planner =   None
action_queue = None


def set_up_envioronment():

    grid = GridMap(width_cells=20, height_cells=15, cell_size=50)
   
    global block_manager, planner, action_queue
    #block_manager = BlockManager(grid)
    #block_manager.add_block("cube1", BlockManager.Block(7, 7, 1, 1))
    planner = PathPlanner(grid)
    action_queue = ActionQueue()

    robot_start = (0, 0)
    robot_angle = 90  # 270 north, 0 east, 90 south, 180 west

    block1_start = (2, 2)  # Where the block is now
    block1_goal = (4, 3)  # Where you want it to go


    block2_start = (2, 0)  # Where the block is now
    block2_goal = (2, 1)  # Where you want it to go




    # 5. Generate ALL commands
    print(f"Generating mission from {block1_start} to {block1_goal}...")
    mission1_queue = planner.generate_mission(robot_start, robot_angle, block1_start, block1_goal)




#    for cmd in mission_queue:
#        action_queue.add(cmd)

    action_queue.add_sequence(mission1_queue)
    print("Final Command Queue:")
    action_queue.print_queue()


In [270]:
set_up_envioronment()

global action_queue
await send_action_queue_event(action_queue.actions)


Generating mission from (2, 2) to (4, 3)...
--- PLANNING MISSION ---
Robot: (0, 0) facing 90
Block: (2, 2) -> (4, 3)
Block Path: [(2, 2), (3, 2), (4, 2), (4, 3)]
Phase 1 (Approach): 4 moves
Phase 2 (Transport): 8 moves
Final Command Queue:
F -> F -> TL -> F -> AB -> F -> F -> TL -> F -> TR -> F -> TR -> F
Sending payload 1/3: [6, 6, 2, 5, 3, 6, 7, 8, 6, 6]
❌ Timeout waiting for action_queue to finish
Sending payload 2/3: [2, 5, 3, 6, 2, 4, 3, 6, 2, 4]
✔ Action queue finished!
Sending payload 3/3: [3, 6, 0, 0, 0, 0, 0, 0, 0, 0]
✔ Action queue finished!
